In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import csv
import re


def pop():
    # 맨 위 뉴스 팝업
    time.sleep(2)
    first = browser.find_element(By.CSS_SELECTOR, '#news-results > div:nth-child(1) > div > div.cont > a > div > strong > span')
    time.sleep(2)
    first.click()

def gettitle(tmp, newsId, newsIdFrom):
    news_title = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.news-view-head > h1.title')
    print('newsTitle:', news_title.text)
    if (len(news_title.text) < 3):
        print("skip!")
        newsId += 1
        newsIdFrom += 1
        return 1
    tmp["newsTitle"].append(news_title.text)
    return 0

def getbody(tmp):
    # 날짜 저장
    news_date = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]')
    for value in news_date:
        print('newsDate:',value.text)
        tmp["newsDate"].append(value.text)
    # 하나의 뉴스기사 기준 본문 저장
    news_bodies = browser.find_elements(By.XPATH, '//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]')
    news_body =''
    for i in news_bodies:
        news_body += i.text
    print('newsBody:',news_body[:50]," ...중략...")
    tmp["newsBody"].append(news_body)

def crawl(newsIdFrom, fromdate, todate):
    # 날짜 구간 설정
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1"]').click()
    time.sleep(2)
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
    time.sleep(2)

    from_ = browser.find_element(By.XPATH, '//*[@id="search-begin-date"]')
    browser.execute_script("arguments[0].value = {};".format(fromdate), from_)
    to_ = browser.find_element(By.XPATH, '//*[@id="search-end-date"]')
    browser.execute_script("arguments[0].value = {};".format(todate), to_)


    # 사건사고 분류 카테고리 - 범죄 선택
    browser.find_element(By.XPATH, '//*[@id="collapse-step-1-body"]/div[3]/div/div[2]/div[3]/a').click()
    time.sleep(2)
    browser.find_element(By.XPATH,'//*[@id="srch-tab4"]/ul/li[1]/div/span[4]').click()
    time.sleep(2)
    browser.find_element(By.XPATH,'//*[@id="search-foot-div"]/div[2]/button[2]').click()
    time.sleep(2)

    # 빈 데이터셋 생성
    tmp = {"newsId":[],"newsTitle":[],"newsDate":[],"newsBody":[]}

    #기간 내 기사를 크롤링
    newsId = 0
    while (newsId < 15000):
        try:
            print('newsId:',newsIdFrom)
            tmp["newsId"].append(newsIdFrom)
            time.sleep(2)
            if (newsId % 10 == 0):
                pop()
            time.sleep(1)
            skip = gettitle(tmp, newsId, newsIdFrom)
            if skip:
                close = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
                time.sleep(1)
                close.click()
                time.sleep(1)
                browser.find_element(By.XPATH, '//*[@id="news-results-tab"]/div[1]/div[2]/div/div/div/div/div[4]/a').click()
                newsId = newsId + ( 10 - newsId % 10 )
                newsIdFrom += 1
                continue
            getbody(tmp)
            time.sleep(2)
            # 다음 뉴스 글로 넘기기
            if (newsId == 0):
                browser.find_element(By.CSS_SELECTOR,
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(2) > dd > a > span').click()
            elif (newsId % 10 == 9 or newsId % 10 == 0 ):
                next_news = browser.find_element(By.CSS_SELECTOR, 
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li:nth-child(3) > dd > a > span')
                time.sleep(1)
                next_news.click()
            else:
                next_news = browser.find_element(By.CSS_SELECTOR,
                    '#news-detail-modal > div > div > div.modal-body > div > div.list_prev_next > ul > li.nextNewsItem > dd > a > span')
                time.sleep(1)
                next_news.click()
            newsId += 1
            newsIdFrom += 1
            # save to csv
            df = pd.DataFrame(tmp)
            filename = "dataset_"+fromdate+"_"+todate+".csv"
            if not os.path.exists(filename):
                df.to_csv(filename, index=False, mode="w", encoding="utf-8-sig")
            else:
                df.to_csv(filename, index=False, mode="a", encoding="utf-8-sig", header=False)
        except:
            print("raise exception")
            newsId += 1
            newsIdFrom += 1
        finally:
            if (newsId > 2 and tmp["newsTitle"][-1] == tmp["newsTitle"][-2] == tmp["newsTitle"][-3] 
                and tmp["newsBody"][-1][:10] == tmp["newsBody"][-2][:10] == tmp["newsBody"][-2][:10]):
                break
    # 창 닫기
    time.sleep(2)
    close = browser.find_element(By.CSS_SELECTOR, '#news-detail-modal > div > div > div.modal-footer > button')
    time.sleep(2)
    close.click()
    time.sleep(2)
    
    print("-"*40,"Complete from {} to {}".format(fromdate, todate),"-"*40)
    return newsId


# selenium
browser = webdriver.Chrome()

browser.set_window_size(1920,1000)
url = "https://www.bigkinds.or.kr/v2/news/search.do"

browser.get(url)
time.sleep(2)


newsId = 0

# 기간 설정
# fromdate = "\'2022-02-28\'"
# todate ="\'2022-02-28\'"
# newsId += crawl(newsId, fromdate, todate)


fromdate = "\'2022-02-27\'"
todate ="\'2022-02-27\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-02-26\'"
todate ="\'2022-02-26\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-02-25\'"
todate ="\'2022-02-25\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-02-24\'"
todate ="\'2022-02-24\'"
newsId += crawl(newsId, fromdate, todate)

fromdate = "\'2022-02-23\'"
todate ="\'2022-02-23\'"
newsId += crawl(newsId, fromdate, todate)

print("-"*40,"Complete to acquisite.","-"*40)

newsId: 0
newsTitle: [차기정부에 바란다] "고용 방식 다양화 등 기업환경 개선 필요"
newsDate: 2022-02-27
newsBody: 손경식 한국경영자총협회 회장
한국경영자총협회(회장 손경식·사진)는 경영권 방어, 조세혁신   ...중략...
newsId: 1
newsTitle: 단연 1순위 `경제 활성화`… `기업가 정신` 존중받아야 새 도약
newsDate: 2022-02-27
newsBody: 보호무역 확산에 러시아·우크라이나 전쟁 '중대재해처벌법' 등 규제 더해져 큰 위기 법인세·  ...중략...
newsId: 2
newsTitle: [사설] 공포의 중대재해법 한 달, 어떻게든 보완 시급하다
newsDate: 2022-02-27
newsBody: 기업에 ‘최악의 중대재해’가 될 것이라던 중대재해처벌법에 대한 우려가 기우(杞憂)가 아니었  ...중략...
newsId: 3
newsTitle: KIA, 한화와 연습경기…투수·외야수 뜨거운 선발 경쟁
newsDate: 2022-02-27
newsBody: 투수 한승혁·유승철·윤중현 외야 이우성·고종욱·김호령 등 김종국 감독에 어필 무대
KIA   ...중략...
newsId: 4
newsTitle: 야구 수업 안전조치 미흡해 유치원생 공 맞아 부상, 선고유예
newsDate: 2022-02-27
newsBody: 잘못 반성하고 합의한 학교장·야구감독
야구 수업을 하면서 사고 예방을 위한 조치를 제대로   ...중략...
newsId: 5
newsTitle: 장성군 감·복숭아·포도·민물장어 지역특산물 지정…6차산업 부흥 기대
newsDate: 2022-02-27
newsBody: 진원면·남면 개발제한구역 내 가공·판매·체험시설 설치 가능
장성군청
장성군이 감·복숭아·포  ...중략...
newsId: 6
newsTitle: [매일춘추] 기착지
newsDate: 2022-02-27
newsBody: 임수현 시인
뉴스를 보다 보면 '신은 없는 게 분명하구나!' 느낄 때가 있다. 신이 아파

newsTitle: 佛 '러 화물선' 나포·獨은 '영공 제한' 합류…바닷길·하늘길 다막는 유럽
newsDate: 2022-02-27
newsBody: 그리스 국적 민간인 10명 사망에
러시아의 우크라이나 침공 이틀째인 25일(현지 시간) 우  ...중략...
newsId: 57
newsTitle: [로터리] 제조업 활성화하려면
newsDate: 2022-02-27
newsBody: 김기준 섬유산업연합회 부회장
지난 24일 한국섬유산업연합회를 비롯해 기계·자동차 등 300  ...중략...
newsId: 58
newsTitle: 전쟁·금리 등 '4대 변수' 속 2500~2800P…"매도보단 보유"
newsDate: 2022-02-27
newsBody: [증권사 3월 증시 전망]
전쟁 공포와 그에 따른 인플레이션 우려에 차갑게 얼어붙었던 증시  ...중략...
newsId: 59
newsTitle: "신용장 발급 이미 막혔다"…자금회수 비상
newsDate: 2022-02-27
newsBody: [한러 무역 올스톱]
독일 프랑크푸르트 시내에서 26일(현지시간) 러시아의 우크라이나 침공  ...중략...
newsId: 60
newsTitle: 러 사업 어쩌나…'우크라 불똥'에 골치 아픈 美 빅테크
newsDate: 2022-02-27
newsBody: ■ 글로벌w '양국 압박에 코너 몰린 애플·구글·메타'
블라디미르 푸틴 러시아 대통령 /A  ...중략...
newsId: 61
newsTitle: [대선 민심 르포] 대구경북, '국론 분열 징그럽다' 통합의 정치 촉구
newsDate: 2022-02-27
newsBody: [한신협 공동기획] 차기 대통령에게 바란다||바닥 치는 경제와 지역 현안 해결 총력 당부하  ...중략...
newsId: 62
newsTitle: “학폭 잡아라”... 中 학교에 ‘공안 교감’ 배치 초강수 [특파원 24시]
newsDate: 2022-02-27
newsBody: 초·중등학교에 사법기관 관계자 교감으로 배치10대 비행 범죄 예방 등

newsId: 112
newsTitle: '횡령·갑질·성희롱' 비리 얼룩진 새마을금고 개혁한다는데…
newsDate: 2022-02-27
newsBody: 횡령, 직장 내 갑질, 성희롱 등 각종 비리가 이어지고 있는 새마을금고를 엄단하겠다며 정부  ...중략...
newsId: 113
newsTitle: 선거 운동원 옷찢고 현수막 훼손... 대구 50대 여성에 영장신청
newsDate: 2022-02-27
newsBody: 경찰로고. /조선DB
대구동부경찰서는 선거운동 중이던 정당 선거사무원을 폭행한 혐의로 50  ...중략...
newsId: 114
newsTitle: "푸틴 최측근 제재, 반도체 수출 통제에 스위프트 퇴출까지"
newsDate: 2022-02-27
newsBody: 벨기에 브뤼셀에 있는 국제은행간통신협회(SWIFT·스위프트) 본부의 로고/사진=AFP
[머  ...중략...
newsId: 115
newsTitle: 경남 간 이재명 “홍준표 지사께 감사…사회적 일자리는 유승민 후보 것”
newsDate: 2022-02-27
newsBody: 이재명 더불어민주당 대선 후보가 27일 경남 창원시 상남분수광장에서 열린 유세에서 지지를   ...중략...
newsId: 116
newsTitle: 러시아 은행, SWIFT서 배제… 한국 대금거래 막히나(종합2보)
newsDate: 2022-02-27
newsBody: [이미지출처=EPA연합뉴스]
[아시아경제 조성필 기자] 국을 비롯한 서방이 26일(현지시간  ...중략...
newsId: 117
newsTitle: 장성군, 지역특산물 지정…그린벨트 규제 해결
newsDate: 2022-02-27
newsBody: '감·복숭아·포도·민물장어' 지역특산물 지정…6차산업 부흥기대
장성군 진원면 불태산 일대
  ...중략...
newsId: 118
newsTitle: 5.18 참여 10대 42년 만에 ‘무죄’
newsDate: 2022-02-27
newsBody: 수원고법 “재심 사유없어 폭력 혐의는 ‘유죄’


newsId: 167
newsTitle: 경찰청, 마약류 사범 집중 단속...내달 1일부터 5월31일까지
newsDate: 2022-02-27
newsBody: "마약류 범죄 검거 인원 지속적으로 증가"
위 사진은 기사 내용과 관련 없습니다.
[사진=  ...중략...
newsId: 168
newsTitle: 보험금 2억 타고자 10년간 전신마비 행세한 모녀...法 "징역 3년"
newsDate: 2022-02-27
newsBody: "보험금 편취하려 전신마비 행세...죄질이 매우 좋지 않아"
위 사진은 기사 내용과 관련   ...중략...
newsId: 169
newsTitle: 무역위, 중국・태국・인니산 폴리아미드 필름 등 반덤핑조사 개시
newsDate: 2022-02-27
newsBody: ⓒ게티이미지뱅크
산업통상자원부 무역위원회가 '중국・태국・인도네시아산 폴리아미드 필름', '  ...중략...
newsId: 170
newsTitle: 김제서 사촌 형수 살해한 50대 속초서 검거
newsDate: 2022-02-27
newsBody: 전북 김제에서 사촌 형수를 살해하고 강원도로 도주한 50대 남성이 경찰에 붙잡혔다.

전북  ...중략...
newsId: 171
newsTitle: 10년간 전신마비 행세… 보험금 2억 타낸 모녀 각각 징역 3년
newsDate: 2022-02-27
newsBody: 서울중앙지법/뉴스1
10년 동안 전신마비 환자인 척 행세하며 허위로 보험금 2억여원을 타낸  ...중략...
newsId: 172
newsTitle: OTT 시즌, 원지안X윤찬영 '소년비행' 선보인다
newsDate: 2022-02-27
newsBody: 오리지널 '소년비행' 25일 오후 6시 공개 '쿠프 드 프랑스' 준결승전도 독점 제공
[아  ...중략...
newsId: 173
newsTitle: “살려 놨더니 주먹질”… 경찰 폭행한 만취 남성들에 징역형
newsDate: 2022-02-27
newsBody: 법원. /조선DB
술에 취해 저수지로 들어

raise exception
newsId: 300
newsTitle: 
skip!
raise exception
newsId: 301
newsTitle: 
skip!
raise exception
newsId: 302
newsTitle: 
skip!
raise exception
newsId: 303
newsTitle: 
skip!
raise exception
newsId: 304
newsTitle: 
skip!
raise exception
newsId: 305
newsTitle: 
skip!
raise exception
newsId: 306
newsTitle: 
skip!
raise exception
newsId: 307
newsTitle: 
skip!
raise exception
newsId: 308
newsTitle: 
skip!
raise exception
newsId: 309
newsTitle: 
skip!
raise exception
newsId: 310
newsTitle: 
skip!
raise exception
newsId: 311
newsTitle: 
skip!
raise exception
newsId: 312
newsTitle: 
skip!
raise exception
newsId: 313
newsTitle: 
skip!
raise exception
newsId: 314
newsTitle: 
skip!
raise exception
newsId: 315
newsTitle: 
skip!
raise exception
newsId: 316
newsTitle: 
skip!
raise exception
newsId: 317
newsTitle: 
skip!
raise exception
newsId: 318
newsTitle: 
skip!
raise exception
newsId: 319
newsTitle: 
skip!
raise exception
newsId: 320
newsTitle: 
skip!
raise exception
newsId: 321
newsTi

...

raise exception
newsId: 657
newsTitle: 
skip!
raise exception
newsId: 658
newsTitle: 
skip!
raise exception
newsId: 659
newsTitle: 
skip!
raise exception
newsId: 660
newsTitle: 
skip!
raise exception
newsId: 661
newsTitle: 
skip!
raise exception
newsId: 662
newsTitle: 
skip!
raise exception
newsId: 663
newsTitle: 
skip!
raise exception
newsId: 664
newsTitle: 
skip!
raise exception
newsId: 665
newsTitle: 
skip!
raise exception
newsId: 666
newsTitle: 
skip!
raise exception
newsId: 667
newsTitle: 
skip!
raise exception
newsId: 668
newsTitle: 
skip!
raise exception
newsId: 669
newsTitle: 
skip!
raise exception
newsId: 670
newsTitle: 
skip!
raise exception
newsId: 671
newsTitle: 
skip!
raise exception
newsId: 672
newsTitle: 
skip!
raise exception
newsId: 673
newsTitle: 
skip!
raise exception
newsId: 674
newsTitle: 
skip!
raise exception
newsId: 675
newsTitle: 
skip!
raise exception
newsId: 676
newsTitle: 
skip!
raise exception
newsId: 677
newsTitle: 
skip!
raise exception
newsId: 678
newsTi

newsTitle: 
skip!
raise exception
newsId: 836
newsTitle: 
skip!
raise exception
newsId: 837
newsTitle: 
skip!
raise exception
newsId: 838
newsTitle: 
skip!
raise exception
newsId: 839
newsTitle: 
skip!
raise exception
newsId: 840
newsTitle: 
skip!
raise exception
newsId: 841
newsTitle: 
skip!
raise exception
newsId: 842
newsTitle: 
skip!
raise exception
newsId: 843
newsTitle: 
skip!
raise exception
newsId: 844
newsTitle: 
skip!
raise exception
newsId: 845
newsTitle: 
skip!
raise exception
newsId: 846
newsTitle: 
skip!
raise exception
newsId: 847
newsTitle: 
skip!
raise exception
newsId: 848
newsTitle: 
skip!
raise exception
newsId: 849
newsTitle: 
skip!
raise exception
newsId: 850
newsTitle: 
skip!
raise exception
newsId: 851
newsTitle: 
skip!
raise exception
newsId: 852
newsTitle: 
skip!
raise exception
newsId: 853
newsTitle: 
skip!
raise exception
newsId: 854
newsTitle: 
skip!
raise exception
newsId: 855
newsTitle: 
skip!
raise exception
newsId: 856
newsTitle: 
skip!
raise exception


newsTitle: 
skip!
raise exception
newsId: 1014
newsTitle: 
skip!
raise exception
newsId: 1015
newsTitle: 
skip!
raise exception
newsId: 1016
newsTitle: 
skip!
raise exception
newsId: 1017
newsTitle: 
skip!
raise exception
newsId: 1018
newsTitle: 
skip!
raise exception
newsId: 1019
newsTitle: 
skip!
raise exception
newsId: 1020
newsTitle: 
skip!
raise exception
newsId: 1021
newsTitle: 
skip!
raise exception
newsId: 1022
newsTitle: 
skip!
raise exception
newsId: 1023
newsTitle: 
skip!
raise exception
newsId: 1024
newsTitle: 
skip!
raise exception
newsId: 1025
newsTitle: 
skip!
raise exception
newsId: 1026
newsTitle: 
skip!
raise exception
newsId: 1027
newsTitle: 
skip!
raise exception
newsId: 1028
newsTitle: 
skip!
raise exception
newsId: 1029
newsTitle: 
skip!
raise exception
newsId: 1030
newsTitle: 
skip!
raise exception
newsId: 1031
newsTitle: 
skip!
raise exception
newsId: 1032
newsTitle: 
skip!
raise exception
newsId: 1033
newsTitle: 
skip!
raise exception
newsId: 1034
newsTitle: 
s

raise exception
newsId: 1188
newsTitle: 
skip!
raise exception
newsId: 1189
newsTitle: 
skip!
raise exception
newsId: 1190
newsTitle: 
skip!
raise exception
newsId: 1191
newsTitle: 
skip!
raise exception
newsId: 1192
newsTitle: 
skip!
raise exception
newsId: 1193
newsTitle: 
skip!
raise exception
newsId: 1194
newsTitle: 
skip!
raise exception
newsId: 1195
newsTitle: 
skip!
raise exception
newsId: 1196
newsTitle: 
skip!
raise exception
newsId: 1197
newsTitle: 
skip!
raise exception
newsId: 1198
newsTitle: 
skip!
raise exception
newsId: 1199
newsTitle: 
skip!
raise exception
newsId: 1200
newsTitle: 
skip!
raise exception
newsId: 1201
newsTitle: 
skip!
raise exception
newsId: 1202
newsTitle: 
skip!
raise exception
newsId: 1203
newsTitle: 
skip!
raise exception
newsId: 1204
newsTitle: 
skip!
raise exception
newsId: 1205
newsTitle: 
skip!
raise exception
newsId: 1206
newsTitle: 
skip!
raise exception
newsId: 1207
newsTitle: 
skip!
raise exception
newsId: 1208
newsTitle: 
skip!
raise excepti

newsTitle: 
skip!
raise exception
newsId: 1363
newsTitle: 
skip!
raise exception
newsId: 1364
newsTitle: 
skip!
raise exception
newsId: 1365
newsTitle: 
skip!
raise exception
newsId: 1366
newsTitle: 
skip!
raise exception
newsId: 1367
newsTitle: 
skip!
raise exception
newsId: 1368
newsTitle: 
skip!
raise exception
newsId: 1369
newsTitle: 
skip!
raise exception
newsId: 1370
newsTitle: 
skip!
raise exception
newsId: 1371
newsTitle: 
skip!
raise exception
newsId: 1372
newsTitle: 
skip!
raise exception
newsId: 1373
newsTitle: 
skip!
raise exception
newsId: 1374
newsTitle: 
skip!
raise exception
newsId: 1375
newsTitle: 
skip!
raise exception
newsId: 1376
newsTitle: 
skip!
raise exception
newsId: 1377
newsTitle: 
skip!
raise exception
newsId: 1378
newsTitle: 
skip!
raise exception
newsId: 1379
newsTitle: 
skip!
raise exception
newsId: 1380
newsTitle: 
skip!
raise exception
newsId: 1381
newsTitle: 
skip!
raise exception
newsId: 1382
newsTitle: 
skip!
raise exception
newsId: 1383
newsTitle: 
s